In [1]:
#load data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))

# exclude data before this unix time : 1356955199 = 12/31/2012
print(len(df))

previousDF = df[df['time'] <= 1356955199]  
# exclude data after this unix time: 
afterDF = df[df['time'] > 1356955199]  
      
print(len(previousDF))
print(len(afterDF))



train_data, test_data = cv.train_test_split(previousDF, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
#print(productDict['B00004XOLO'])
#print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")



Number of users = 826767 | Number of items = 50210
1324753
739884
584869
<bound method lil_matrix.getnnz of <457370x38568 sparse matrix of type '<class 'numpy.float32'>'
	with 739884 stored elements in LInked List format>>
done


In [2]:
# read metadata
import pandas as pd
import json
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i+= 1
    return pd.DataFrame.from_dict(df, orient='index')

metaDF = getDF('meta_Video_Games.json.gz')
# metaDF = pd.read_json("meta_Video_Games.json", orient='split')
print(metaDF)

             asin                                        description   price  \
0      0078764343                                  Brand new sealed!   37.98   
1      043933702X  In Stock NOW.  Eligible for FREE Super Saving ...   23.50   
2      0439339987  Grandma Groupers kelp seeds are missing and wi...    8.95   
3      0439342260  This software is BRAND NEW. Packaging may diff...     NaN   
4      0439339960               a scholastic clubs fairs cd rom game     NaN   
5      0439374391  CD-ROM: Thomas & Friends: The Great Festival A...   20.00   
6      0439394422  Product that encourages families to learn, exp...   12.96   
7      043940133X  Your ship has crash-landed on the planet Tatoo...   30.00   
8      0439573947                      3 Great games in one box set!   10.79   
9      0439591295                                    cartoon network     NaN   
10     0439591368  Travel to a remarkable new world and treat you...   19.95   
11     0439591538  WOW! 3d NANCY DREW MY

In [13]:
import requests
from bs4 import BeautifulSoup

# used when Nan not in metadata, note slower to get names so use at last step
def getProductTitleScrape(id):
    URL = "https://www.amazon.com/dp/" + id
    #print(URL)
    page = requests.get(URL,headers={"User-Agent":"Defined"})
    soup = BeautifulSoup(page.content, "html.parser")
    #print(soup)
    title = soup.find(id="productTitle").get_text().strip()
    print(title)

getProductTitleScrape("0439715571")



Mortal Kombat 4


In [4]:
#print(metaDF)
query = metaDF.loc[metaDF['asin'] == "B000007VDN"]
title = query['title'].iloc[0]
print(title)

def getProductTitle(asin):
    query = metaDF.loc[metaDF['asin'] == asin]
    title = str(query['title'].iloc[0])
    if title == "nan":
        print("Title not availiable for product " + str(asin))
    else:
        print(title)


getProductTitle("B003JKK2JC")


Underworld
Title not availiable for product B003JKK2JC


In [5]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat.T * col_normed_mat

def user_cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=1)
    #print(col_normed_mat)
    return col_normed_mat * col_normed_mat.T


#cosine_similarities(mat)
test = cosine_similarities(mat)
#userMatrix = user_cosine_similarities(mat)
#user_similarity = pairwise_distances(mat, metric='cosine')
print("done")
print(test.getnnz)
#print(userMatrix.getnnz)


done
<bound method _cs_matrix.getnnz of <38568x38568 sparse matrix of type '<class 'numpy.float32'>'
	with 5776718 stored elements in Compressed Sparse Row format>>


In [6]:
# https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
#print(test[50208,40340])

#print(userMatrix.getrow(826766))

print("============")

row = test.getrow(6565).toarray()[0].ravel()
top_ten_indicies = row.argsort()[-6:]
top_ten_values = row[row.argsort()[-6:]]

print(top_ten_indicies)

print(top_ten_values)

#for x in top_ten_indicies:
#    print(list(productDict.keys())[list(productDict.values()).index(x)])
    


#for x in range(0,n_items):
#    print(x)
#    row = test.getrow(x).toarray()[0].ravel()
#    top_ten_values = (row[row.argsort()[-10:]])

print("done")



#array = test.getrow(i)
#print(array)


[34481 38140 38303 35891 32959  6565]
[0.17835118 0.22293898 0.22293898 0.22293898 0.22293898 1.0000001 ]
done


In [11]:
def getTopFiveSimiliarItems(itemId):
    row = test.getrow(itemId).toarray()[0].ravel()
    top_five_indicies = row.argsort()[-6:]
    top_five_values = row[row.argsort()[-6:]]
    print(top_five_values)
    print(top_five_indicies)
    for x in top_five_indicies:
        #ignore same item
        if (x != itemId):
            print(list(productDict.keys())[list(productDict.values()).index(x)])
            #getProductTitleScrape(list(productDict.keys())[list(productDict.values()).index(x)])
            #id = list(productDict.keys())[list(productDict.values()).index(x)]
            #print(getProductTitle(id))
        
#getTopFiveSimiliarItems(9331)
#getTopFiveSimiliarItems(productDict["B000EBC0PI"])
    

In [12]:
print(df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"])
print("space")
line = df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"]

for x in line.iterrows():
    #print(x[1][1])
    print(productDict[x[1][1]])
    getTopFiveSimiliarItems(productDict[x[1][1]])
    #print("======================")
    
df.loc[df['reviewerId'] == "A10RFAF1CGFKHZ"]
print(list(productDict.keys())[list(productDict.values()).index(9331)]) 
print(list(productDict.keys())[list(productDict.values()).index(23340)]) 
print(list(userDict.keys())[list(userDict.values()).index(76565)]) 

            reviewerId   productId  rating        time
124881  A2UDD2XDPD80BH  B000066TS5     1.0  1160956800
198242  A2UDD2XDPD80BH  B0001I1LD6     5.0  1145664000
204696  A2UDD2XDPD80BH  B00020LZAW     5.0  1142553600
228327  A2UDD2XDPD80BH  B00067AVT2     5.0  1142208000
237315  A2UDD2XDPD80BH  B0007PLKZM     5.0  1160092800
258699  A2UDD2XDPD80BH  B0009WPZM2     5.0  1159142400
270903  A2UDD2XDPD80BH  B000B43OXU     5.0  1144540800
284015  A2UDD2XDPD80BH  B000BI3QK2     5.0  1161648000
285552  A2UDD2XDPD80BH  B000BLM5PG     5.0  1156809600
289093  A2UDD2XDPD80BH  B000C427NM     5.0  1155427200
289669  A2UDD2XDPD80BH  B000CBVC0Y     5.0  1170979200
304679  A2UDD2XDPD80BH  B000FBMITA     5.0  1149206400
319037  A2UDD2XDPD80BH  B000FRS9II     5.0  1167436800
space
2245
[0.05767148 0.06042429 0.06479723 0.07220671 0.11668522 0.99999183]
[ 409  517  126 2080 1532 2245]
B00005BW7Q
B00008URUA
B00005TNI6
B00068O278
B0009A4EVM
34796
[0.08928116 0.10481282 0.11155158 0.12192657 0.19233046 1.